In [ ]:
from pathlib import Path
import sys

# Add base dir to path to be able to access test folder.
base_dir = Path().cwd().parent.parent.resolve()
if str(base_dir) not in sys.path:
    print(f"Adding {base_dir} to sys.path")
    sys.path.append(str(base_dir))

In [ ]:
import logging

root = logging.getLogger()
root.setLevel(logging.DEBUG)
handler = logging.StreamHandler(sys.stdout)
handler.setLevel(logging.DEBUG)
formatter = logging.Formatter(
    "%(asctime)s - %(name)s - %(levelname)s - %(message)s"
)
handler.addFilter(logging.Filter("trulens"))
handler.setFormatter(formatter)
root.addHandler(handler)

In [ ]:
from trulens.experimental.otel_tracing.core.instrument import instrument
from trulens.otel.semconv.trace import SpanAttributes


class TestApp:
    @instrument(span_type=SpanAttributes.SpanType.MAIN)
    def respond_to_query(self, query: str) -> str:
        return f"answer: {self.nested(query)}"

    @instrument(attributes={"nested_attr1": "value1"})
    def nested(self, query: str) -> str:
        return f"nested: {self.nested2(query)}"

    @instrument(
        attributes=lambda ret, exception, *args, **kwargs: {
            "nested2_ret": ret,
            "nested2_args[1]": args[1],
        }
    )
    def nested2(self, query: str) -> str:
        nested_result = ""

        try:
            nested_result = self.nested3(query)
        except Exception:
            pass

        return f"nested2: {nested_result}"

    @instrument(
        attributes=lambda ret, exception, *args, **kwargs: {
            "nested3_ex": exception.args if exception else None,
            "nested3_ret": ret,
            "selector_name": "special",
            "cows": "moo",
        }
    )
    def nested3(self, query: str) -> str:
        if query == "throw":
            raise ValueError("nested3 exception")
        return "nested3"

In [ ]:
import os

import dotenv
from trulens.core.experimental import Feature
from trulens.core.session import TruSession

dotenv.load_dotenv()
os.environ["TRULENS_OTEL_TRACING"] = "1"
session = TruSession(experimental_feature_flags={Feature.OTEL_TRACING: True})

In [ ]:
from trulens.apps.custom import TruCustomApp

test_app = TestApp()
custom_app = TruCustomApp(
    test_app,
    app_name="database.schema.app",
    app_version="1.0.0",
    session=session,
)

with custom_app() as recording:
    test_app.respond_to_query("test")

with custom_app(run_name="test run", input_id="456") as recording:
    test_app.respond_to_query("throw")

In [ ]:
session.experimental_force_flush()